In [21]:
require 'optim'
require 'io'
require 'torch'
require 'nn'
require 'rnn'
require 'csvigo'
require 'cutorch'
require 'cunn'
require 'cunnx'

dl = require 'dataload'
cmd = torch.CmdLine()

cmd:option('--nepochs', 5, 'running for 50 epochs')
cmd:option('--learning_rate', 1e-5, 'using a learning rate of 1e-5')
cmd:option('--gamma', 0., 'Discount rate parameter in backprop step')
cmd:option('--epsilon', 1., 'Random search rate')
cmd:option('--cuts', 4, 'Discount rate parameter in backprop step')
cmd:option('--base_explore_rate', 0.0, 'Base rate')
cmd:option('--mem_size', 100, 'Memory size')
cmd:option('--batch_size', 200,'Batch Size')
cmd:option('--model','bow','BOW/LSTM option')
cmd:option('--embeddingSize', 64,'Embedding dimension')
cmd:option('--usecuda', false, 'running on cuda')
cmd:option('--metric', "f1", 'Metric to learn')
cmd:option('--n_samples', 20, 'Number of samples to use')
cmd:option('--maxSummarySize', 300, 'Maximum summary size')
cmd:option('--end_baserate', 5, 'Epoch number at which the base_rate ends')
cmd:option('--K_tokens', 25, 'Maximum number of tokens for each sentence')
cmd:option('--thresh', 0, 'Threshold operator')
cmd:option('--adapt', false, 'Domain Adaptation Regularization')
cmd:option('--n_backprops', 3, 'Number of times to backprop through the data')
cmd:text()
--- this retrieves the commands and stores them in opt.variable (e.g., opt.model)
local opt = cmd:parse(arg or {})

dofile("utils.lua")
dofile("utilsNN.lua")

input_path = '~/GitHub/DeepNLPQLearning/DO_NOT_UPLOAD_THIS_DATA/0-output/'

query_fn = input_path .. 'queries_numtext.csv'
query_file =  csvigo.load({path = query_fn, mode = "large", verbose = false})
queries = padZeros(buildTermDocumentTable(query_file, nil), 5)

stoplist = input_path .. 'stopwordids.csv'
stopfile = csvigo.load({path = stoplist, mode = "large", verbose = false})
stoplist = buildTermDocumentTable(stopfile, nil)
stopwords = {}
for k,v in pairs(stoplist) do 
    stopwords[k]  = v[1]
end

local pakistan = {
        ['inputs'] = '2012_pakistan_garment_factory_fires_first_sentence_numtext2.csv',
        ['nuggets'] ='pakistan_nuggets_numtext.csv',
        ['query'] = queries[2],
        ['query_name'] = 'pakistan'
}
local aurora = {
        ['inputs'] = '2012_aurora_shooting_first_sentence_numtext2.csv', 
        ['nuggets'] = 'aurora_nuggets_numtext.csv',
        ['query'] = queries[3],
        ['query_name'] = 'aurora'
}
local wisconsin = {
        ['inputs'] = 'wisconsin_sikh_temple_shooting_first_sentence_numtext.csv',
        ['nuggets'] ='wisconsin_nuggets_numtext.csv',
        ['query'] = queries[4],
        ['query_name'] = 'wisconsin'
}
local sandy = {
        ['inputs'] = 'hurricane_sandy_first_sentence_numtext2.csv',
        ['nuggets'] ='sandy_nuggets_numtext.csv',
        ['query'] = queries[6],
        ['query_name'] = 'sandy'
}

inputs = {
        aurora, 
        pakistan,
}

if opt.usecuda then
    Tensor = torch.CudaTensor
    LongTensor = torch.CudaLongTensor
    ByteTensor = torch.CudaByteTensor
    print("...running on GPU")
else
    torch.setnumthreads(8)
    Tensor = torch.Tensor
    LongTensor = torch.LongTensor
    ByteTensor = torch.ByteTensor
    print("...running on CPU")
end

local delta = opt.cuts/opt.nepochs
local optimParams = { learningRate = opt.learning_rate }

-- Initializing the model variables
vocabSize, query_data = intialize_variables(query_file, inputs, 
                                            opt.n_samples, input_path, opt.K_tokens, 
                                            opt.maxSummarySize, stopwords, opt.thresh)

...Utils file loaded	


...running on CPU	


In [22]:
refCounts = query_data[1][12]

In [23]:
refSummary = query_data[1][11]

In [24]:
refCounts2 = buildTokenCounts(refSummary, nil)

In [25]:
refCounts

{
  2 : 0
  6 : 0
  7 : 0
  10 : 0
  15 : 0
  828 : 0
  234 : 0
  245 : 0
  1285 : 0
  500 : 0
  778 : 0
  502 : 0
  782 : 0
  1054 : 0
  774 : 0
  1223 : 0
  60 : 0
  1506 : 0
  1518 : 0
  1673 : 0
  1426 : 0
  1422 : 0
  51 : 0
  405 : 0
  406 : 0
  1466 : 0
  715 : 0
  402 : 0
  403 : 0
  412 : 0
  413 : 0
  205 : 0
  408 : 0
  203 : 0
  411 : 0
  958 : 139
  196550 : 556
  689 : 0
  426 : 0
  669 : 0
  665 : 0
  1286 : 0
  663 : 0
  1027 : 0
  1043 : 0
  1047 : 0
  1051 : 0
  223 : 0
  1503 : 0
  1224 : 0
  1220 : 0
  889 : 0
  25 : 0
  1511 : 0
  1507 : 0
  166 : 0
  618 : 0
  616 : 0
  614 : 0
  85 : 0
  1423 : 0
  1419 : 0
  1698 : 0
  839 : 0
  1447 : 0
  336 : 0
  1120 : 0
  348 : 0
  1395 : 0
  1068 : 0
  1339 : 0
  1028 : 0
  781 : 0
  1044 : 0
  1048 : 0
  771 : 0
  773 : 0
  775 : 0
  1225 : 0
  1221 : 0
  136 : 0
  1512 : 0
  1675 : 0
  44 : 0
  1456 : 0
  1452 : 0
  1448 : 0
  133 : 0
  642 : 0
  428 : 0


  64 : 0
  1670 : 0
  1676 : 0
  1671 : 0
  1672 : 0
  662 : 0
  1421 : 0
  383 : 0
  81 : 0
  919 : 139
  1425 : 0
  670 : 0
  664 : 0
  1280 : 0
  1284 : 0
  311 : 0
  776 : 0
  309 : 0
  1320 : 0
  1041 : 0
  1045 : 0
  1049 : 0
  619 : 0
  236 : 0
  157 : 0
  615 : 0
  156 : 139
  666 : 0
  450 : 0
  1222 : 0
  312 : 0
  252 : 0
  153 : 0
  404 : 0
  625 : 0
  230 : 0
  1517 : 0
  1513 : 0
  74 : 0
  17 : 0
  1666 : 139
  620 : 0
  1052 : 0
  314 : 0
  466 : 0
  1055 : 0
  1046 : 0
  465 : 0
  1040 : 0
  1016 : 0
  1445 : 139
  1042 : 0
  204 : 0
  68 : 0
  206 : 0
  222 : 0
  233 : 0
  1424 : 0
  57 : 0
}


In [26]:
refCounts2

{
  444 : 139
  958 : 139
  196550 : 556
  24 : 139
  1445 : 139
  20 : 139
  1117 : 139
  9 : 139
  1666 : 139
  919 : 139
  156 : 139
}


In [19]:
out =0 
for k,v in pairs(refCounts) do out = out +  v end

In [20]:
out

1946	


In [11]:
#refSummary

    1
 1946
[torch.LongStorage of size 2]



In [13]:
refCounts

{
  2 : 0
  6 : 0
  7 : 0
  10 : 0
  15 : 0
  828 : 0
  234 : 0
  245 : 0
  1285 : 0
  500 : 0
  778 : 0
  502 : 0
  782 : 0
  1054 : 0
  774 : 0
  1223 : 0
  60 : 0
  1506 : 0
  1518 : 0
  1673 : 0
  1426 : 0
  1422 : 0
  51 : 0
  405 : 0
  406 : 0
  1466 : 0
  715 : 0
  402 : 0
  403 : 0
  412 : 0
  413 : 0
  205 : 0
  408 : 0
  203 : 0
  411 : 0
  958 : 139
  196550 : 556
  689 : 0
  426 : 0
  669 : 0
  665 : 0
  1286 : 0
  663 : 0
  1027 : 0
  1043 : 0
  1047 : 0
  1051 : 0
  223 : 0
  1503 : 0
  1224 : 0
  1220 : 0
  889 : 0
  25 : 0
  1511 : 0
  1507 : 0
  166 : 0
  618 : 0
  616 : 0
  614 : 0
  85 : 0
  1423 : 0
  1419 : 0
  1698 : 0
  839 : 0
  1447 : 0
  336 : 0
  1120 : 0
  348 : 0
  1395 : 0
  1068 : 0
  1339 : 0
  1028 : 0
  781 : 0
  1044 : 0
  1048 : 0
  771 : 0
  773 : 0
  775 : 0
  1225 : 0
  1221 : 0
  136 : 0
  1512 : 0
  1675 : 0
  44 : 0
  1456 : 0
  1452 : 0
  1448 : 0
  133 : 0
  642 : 0
  428 : 0


  64 : 0
  1670 : 0
  1676 : 0
  1671 : 0
  1672 : 0
  662 : 0
  1421 : 0
  383 : 0
  81 : 0
  919 : 139
  1425 : 0
  670 : 0
  664 : 0
  1280 : 0
  1284 : 0
  311 : 0
  776 : 0
  309 : 0
  1320 : 0
  1041 : 0
  1045 : 0
  1049 : 0
  619 : 0
  236 : 0
  157 : 0
  615 : 0
  156 : 139
  666 : 0
  450 : 0
  1222 : 0
  312 : 0
  252 : 0
  153 : 0
  404 : 0
  625 : 0
  230 : 0
  1517 : 0
  1513 : 0
  74 : 0
  17 : 0
  1666 : 139
  620 : 0
  1052 : 0
  314 : 0
  466 : 0
  1055 : 0
  1046 : 0
  465 : 0
  1040 : 0
  1016 : 0
  1445 : 139
  1042 : 0
  204 : 0
  68 : 0
  206 : 0
  222 : 0
  233 : 0
  1424 : 0
  57 : 0
}
